Observa cómo modelamos con una clase Python un robot en 2D con orientación. Esta clase la usamos para representar nuestro robot y también para representar las partículas

In [1]:
from math import *
import matplotlib.pyplot as plt
%matplotlib inline
from random import gauss
import numpy as np
import bisect

%run -i code/filter.py

class RobotPosition:
    def __init__(self, landmarks, world_size):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 1.0;
        self.landmarks  = landmarks
        self.world_size = world_size
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= self.world_size:
            raise ValueError, 'X coordinate out of bound'
        if new_y < 0 or new_y >= self.world_size:
            raise ValueError, 'Y coordinate out of bound'
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(self.landmarks)):
            dist = sqrt((self.x - self.landmarks[i][0]) ** 2 + (self.y - self.landmarks[i][1]) ** 2)
            dist += gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError, 'Robot cant move backwards'         
        
        # turn, and add randomness to the turning command
        orientation = (self.orientation + float(turn) + gauss(0.0, self.turn_noise)) % (2*pi)
        
        # move, and add randomness to the motion command
        dist = float(forward) + gauss(0.0, self.forward_noise)
        x = ( self.x + (cos(orientation) * dist) ) % self.world_size
        y = ( self.y + (sin(orientation) * dist) ) % self.world_size
        
        # set particle
        res = RobotPosition(self.landmarks, self.world_size)
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        res.landmarks = self.landmarks
        res.world_size = self.world_size
        return res
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

Observa cómo creamos un robot y tratamos de medir su distancia a las balizas

In [2]:
lmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
wsize   = 100.0


myrobot = RobotPosition(lmarks, wsize)
myrobot.x, myrobot.y = 70,70
myrobot.sense_noise  =2.
print myrobot
print myrobot.sense()
print myrobot.sense()
print myrobot.sense()

[x=70 y=70 orient=1.4816]
[71.35934877051452, 15.03244964879969, 51.63808950221352, 53.68584911858603]
[71.11860899068247, 14.425939762840919, 48.019271284512286, 52.98692072127411]
[70.16466735055954, 13.799581006113863, 48.44660909702611, 54.62543264154658]


### Problema 1

Crea una función que para una partícula y una medida, devuelva la probabilidad de que la partícula haya realizado esa medida, según lo descrito en las notas de la lección ([aquí](Notas%2003C%20-%20Filtros%20de%20Partículas.ipynb))

El argumento `particula` es una instancia de la clase `RobotPosition`. La clase tiene un atributo `landmarks` que es una lista de tuplas con las posiciones de las balizas. El argumento `measurement` es una lista con tantos elementos como balizas y contiene la medida realizada por un robot a cada una de las balizas.

**Ejemplo de ejecución**

Entrada:

    lmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
    wsize   = 100.0

    myrobot = RobotPosition(lmarks, wsize)
    myrobot.x, myrobot.y = 70,70
    myrobot.sense_noise  = 5.
    measurement = myrobot.sense()
    print myrobot

    p = RobotPosition(lmarks, wsize)
    p.x, p.y = myrobot.x, myrobot.y
    p.sense_noise = myrobot.sense_noise
    print p
    print measurement_prob(p,measurement)

    for i in range(10):
        p = RobotPosition(lmarks, wsize)
        p.sense_noise = myrobot.sense_noise
        prob = measurement_prob(p,measurement)
        print p, prob
    
    
Salida (los valores serán distintos pero de magnitud similar):

    [x=70 y=70 orient=0.5939]
    [x=70 y=70 orient=4.7417]
    1.20739526925e-05
    [x=35.058 y=22.766 orient=2.5137] 1.57800255111e-70
    [x=73.985 y=89.160 orient=0.0581] 4.68426868039e-09
    [x=37.750 y=77.126 orient=1.0866] 4.58135003998e-25
    [x=58.429 y=90.984 orient=0.8507] 1.09546626438e-10
    [x=94.533 y=50.080 orient=0.1796] 3.87269033669e-24
    [x=11.787 y=99.159 orient=0.0627] 5.16256080313e-64
    [x=38.201 y=4.0926 orient=5.9957] 3.59484963092e-86
    [x=40.393 y=94.338 orient=6.1878] 2.11978338183e-25
    [x=30.119 y=20.642 orient=2.6737] 7.55580454417e-81
    [x=96.038 y=96.639 orient=5.1351] 1.28879112854e-29

In [3]:
import numpy as np

def Gaussian(mu, sigma, x):
    # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
    return np.exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / np.sqrt(2.0 * pi * (sigma ** 2))


def measurement_prob(particle, measurement):

    # calculates how likely a measurement should be
    prob = 1.0;
   
    for i in range(len(particle.landmarks)):
        # ----- TU CODIGO AQUI        
        #
        mu= sqrt((particle.x - particle.landmarks[i][0]) ** 2 + (particle.y - particle.landmarks[i][1]) ** 2)
        sigma=particle.sense_noise;
        prob*=Gaussian(mu,sigma,measurement[i])
        # -----
    return prob
        

Comprueba tu código

In [4]:
lmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
wsize   = 100.0

myrobot = RobotPosition(lmarks, wsize)
myrobot.x, myrobot.y = 70,70
myrobot.sense_noise  = 5.
measurement = myrobot.sense()
print myrobot

p = RobotPosition(lmarks, wsize)
p.x, p.y = myrobot.x, myrobot.y
p.sense_noise = myrobot.sense_noise
print p
print measurement_prob(p,measurement)

for i in range(10):
    p = RobotPosition(lmarks, wsize)
    p.sense_noise = myrobot.sense_noise
    prob = measurement_prob(p,measurement)
    print p, prob
    

[x=70 y=70 orient=4.0076]
[x=70 y=70 orient=1.8228]
7.95459519075e-07
[x=63.217 y=32.464 orient=1.5793] 3.50010018904e-26
[x=80.141 y=69.490 orient=3.5122] 1.77867206281e-09
[x=88.873 y=91.589 orient=1.5522] 4.49990731051e-28
[x=95.385 y=25.687 orient=1.1494] 9.7678928206e-44
[x=63.285 y=7.8584 orient=1.6027] 4.62118229529e-55
[x=29.244 y=1.4630 orient=6.0359] 9.85739550864e-89
[x=36.726 y=60.943 orient=6.1405] 2.53780104168e-25
[x=3.9028 y=38.034 orient=2.7556] 1.86528895893e-78
[x=82.776 y=96.490 orient=6.1914] 1.50663246218e-28
[x=94.159 y=24.455 orient=3.6516] 1.24940717098e-44



### Problema 2

completa la función `weighted_sample` para que dada una lista de pesos, muestree el índice de cada peso de manera proporcional al peso.

**Ejemplo de ejecución**

Entrada:

    weights = np.array([0.8,0.2])
    print weighted_sample(weights, n_samples=20)
    print "--"

    weights = np.array([0.1,0.3,0.4,0.05,0.1,0.05])
    w = weighted_sample(weights, n_samples=10000)
    print "idx freq"
    for i in np.unique(w):
        print "%2d  %.3f"%(i, np.sum(w==i)*1./len(w))


Salida (los valores serán distintos pero de magnitud similar):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
    --
    idx freq
     0  0.099
     1  0.298
     2  0.402
     3  0.050
     4  0.101
     5  0.050


In [7]:
import numpy as np
def weighted_sample(weight_list, n_samples=None):    
    if n_samples is None:
        n_samples=len(weight_list)
    
   
    # -- TU CODIGO AQUI ---
    particulas=np.random.rand(n_samples)
    result =np.zeros(n_samples).astype(int)
    a=0
    for i in range (len(weight_list)):
        b=np.sum(weight_list[0:(i+1)])
        result+=(particulas<b)*(particulas>a)*int(i)
         
#         print"a=",a,"b=",b
        a=b    
    return list(result)


Comprueba tu código

In [10]:

weights = np.array([0.8,0.2])
w= weighted_sample(weights, n_samples=20)
print w
print "idx freq"
for i in np.unique(w):
    print "%2d  %.3f"%(i, np.sum(w==i)*1./len(w))
print "--"

weights = np.array([0.1,0.3,0.4,0.05,0.1,0.05])
w = weighted_sample(weights, n_samples=10000)
print "idx freq"
for i in np.unique(w):
    print "%2d  %.3f"%(i, np.sum(w==i)*1./len(w))


[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
idx freq
 0  0.900
 1  0.100
--
idx freq
 0  0.102
 1  0.305
 2  0.402
 3  0.051
 4  0.094
 5  0.046
